## Vector DB Ingestion
<img src="https://docs.langchain4j.dev/img/logo.svg" alt="LangChain4J" width="200" height="200">

In [4]:
%use dataframe
%useLatestDescriptors
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import dev.langchain4j.data.segment.TextSegment
import dev.langchain4j.store.embedding.EmbeddingStore
import dev.langchain4j.store.embedding.pgvector.PgVectorEmbeddingStore
import dev.langchain4j.model.embedding.*
import dev.langchain4j.model.embedding.onnx.allminilml6v2.AllMiniLmL6V2EmbeddingModel
import dev.langchain4j.data.embedding.*
import dev.langchain4j.store.embedding.*
import dev.langchain4j.store.embedding.filter.MetadataFilterBuilder.metadataKey
import dev.langchain4j.data.document.Document
import dev.langchain4j.data.document.Metadata
import java.util.Arrays
import java.util.concurrent.atomic.AtomicInteger
import org.jetbrains.kotlinx.dataframe.codeGen.generateCode
import org.jetbrains.kotlinx.dataframe.api.*
import org.jetbrains.kotlinx.dataframe.io.*
import java.sql.DriverManager
import kotlin.Metadata as KotlinMetadata
import dev.langchain4j.model.openai.OpenAiEmbeddingModel
import dev.langchain4j.model.openai.OpenAiEmbeddingModelName
import java.util.UUID

import io.kotest.matchers.comparables.shouldBeGreaterThanOrEqualTo
import io.kotest.matchers.shouldBe
import io.kotest.matchers.shouldNotBe
import io.kotest.matchers.string.shouldStartWith
import dev.langchain4j.data.document.splitter.DocumentSplitters


In [2]:
val host = "localhost"
val port = 5430
val user = "user"
val password = "password"
val database = "vector_store"
val jdbcUrl = "jdbc:postgresql://$host:$port/$database"
val dbConfig = DatabaseConfiguration(jdbcUrl, user, password)
val tableName = "talks"
val mapper = jacksonObjectMapper()

// Test Connection
DataFrame.readSqlQuery(dbConfig, "SELECT table_name FROM information_schema.tables").size().nrow  shouldBeGreaterThanOrEqualTo 0
println("Connection to database successful")

Connection to database successful


## 📘 Ingestion for exercises
#### Prepare Dataframe from: [dataset-kotlinconf.json](../main/resources/dataset-kotlinconf.json).


In [22]:
val df = DataFrame.readJson("./data/dataset-jfall.json")
val sessions = df.explode("sessions").select("sessions").rename("sessions").into("session").flatten()
//generateCode enables us to access the individual columns in a type safe way, rather than Strings
sessions.generateCode()
//this shows the schema of the sessions
sessions.schema()

title: String
startsAt: String
endsAt: String
category: List<String>
room: String
speakers: List<String>
description: String

#### Execute Ingestion precedure

In [14]:
val startsAt = sessions.map { java.time.Instant.parse(startsAt).toEpochMilli() }.first()
startsAt

1762412400000

In [11]:
val openAIApiToken = System.getenv("OPENAI_API_KEY") ?: error("OPENAI_API_KEY environment variable not set")
val embeddingModel:EmbeddingModel = OpenAiEmbeddingModel.builder()
    .apiKey(openAIApiToken)
    .modelName(OpenAiEmbeddingModelName.TEXT_EMBEDDING_3_SMALL)
    .build()

// define EmbeddingStore
val embeddingStore: EmbeddingStore<TextSegment> = PgVectorEmbeddingStore.builder()
    .host(host)
    .port(port)
    .user(user)
    .password(password)
    .database(database)
    .table(tableName)
    .dimension(embeddingModel.dimension())
    .dropTableFirst(true)
    .build();



In [15]:

val zone = java.time.ZoneId.systemDefault()
val conferenceDaysSorted = sessions
    .map { java.time.Instant.parse(it.startsAt).atZone(zone).toLocalDate() }
    .distinct()
    .sorted()
val dayIndexByDate = conferenceDaysSorted.mapIndexed { idx, date -> date to (idx + 1) }.toMap()

fun phaseOfDayFrom(hour: Int, durationMinutes: Long): String = when {
    durationMinutes >= 8 * 60 -> "ALL_DAY"
    hour in 5..11 -> "MORNING"
    hour in 12..17 -> "AFTERNOON"
    else -> "EVENING"
}



//AllMiniLmL6V2EmbeddingModel() //("Use AllMiniLmL6V2EmbeddingModel")


// define ingestor
val ingestor:EmbeddingStoreIngestor = EmbeddingStoreIngestor.builder()
    .documentSplitter(DocumentSplitters.recursive(200, 40))
    .embeddingModel(embeddingModel)
    .embeddingStore(embeddingStore)
    .build()

//: produce list of Documents each containing the session title
val documents:List<Document> = sessions.map {
    val startZoned = java.time.Instant.parse(startsAt).atZone(zone)
    val endZoned = java.time.Instant.parse(endsAt).atZone(zone)
    val startLocal = startZoned.toLocalDateTime()
    val endLocal = endZoned.toLocalDateTime()
    val conferenceDay = startZoned.toLocalDate()
    val durationMinutes = java.time.Duration.between(startZoned, endZoned).toMinutes()
    val phaseOfDay = phaseOfDayFrom(startLocal.hour, durationMinutes)

    Document.from("title:$title, description:$description",
        dev.langchain4j.data.document.Metadata(mapOf<String, Any>(
            "id" to UUID.randomUUID().toString().replace("-", ""),
            "title" to title,
            "room" to room,
            "category" to category.joinToString(),
            "category" to speakers.joinToString(),
            // Extra derived metadata in local date-time (no UTC suffix)
            "startsAtLocalDateTime" to java.time.format.DateTimeFormatter.ISO_LOCAL_DATE_TIME.format(startLocal),
            "endsAtLocalDateTime" to java.time.format.DateTimeFormatter.ISO_LOCAL_DATE_TIME.format(endLocal),
            "startsAt" to java.time.Instant.parse(startsAt).toEpochMilli(),
            "endsAt" to java.time.Instant.parse(endsAt).toEpochMilli(),
            "conferenceDay" to conferenceDay.toString(),
            "dayIndex" to (dayIndexByDate[conferenceDay] ?: 1),
            "phaseOfDay" to phaseOfDay,
            "durationMinutes" to durationMinutes.toInt(),
        ))
    )
}

// execute ingestion
val ingestionResult = ingestor.ingest(documents)

println("Ingested a total of ${documents.size} documents with a total token count of: ${ingestionResult.tokenUsage().totalTokenCount()}")


Ingested a total of 71 documents with a total token count of: 4178


#### Verify Ingestion


In [17]:
val dbDf = DataFrame.readSqlTable(dbConfig, tableName)
dbDf.select( "text", "embedding", "embedding_id", "metadata").head(10)

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-184549368"></table>

<p class="dataframe_description">DataFrame: rowsCount = 10, columnsCount = 4</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i <

In [20]:
//verify db


val searchQuery:String = "Sessions about Agentic AI and MCP Model context Protocol"
// embed the search
val searchQueryEmbedding = embeddingModel.embed(searchQuery).content()

// create the search request with metadata for: `category` containsString `Server-side`
val request: EmbeddingSearchRequest = EmbeddingSearchRequest.builder()
    .queryEmbedding(searchQueryEmbedding)
    .maxResults(5)
    .minScore(0.5)
    //.filter(metadataKey("category").containsString("Server-side"))
    .build()


// perform the search via the EmbeddingStore
println("Looking for talks similar to: $searchQuery...\n")
val relevant:List<EmbeddingMatch<TextSegment>> = embeddingStore.search(request).matches()
//relevant.forEach{
//    println("Matched text: ${it.embedded().text()} with similarity score: ${it.score()}")
//}

val res = relevant.groupBy { it.embedded().metadata().getString("title") }.map { (_, sameDocs) ->
    sameDocs.sortedByDescending { it.score() }.first()
}.also { println("Unique titles found: \n-\t${it.joinToString("\n-\t") { it.embedded().metadata().getString("title").toString()}}") }



Looking for talks similar to: Sessions about Agentic AI and MCP Model context Protocol...

Unique titles found: 
-	Supercharge your LLM with Java: Model Context Protocol (MCP) in Action
-	Level Up Your LangChain4j Apps for Production
-	From Scratch to Scalable: Building Smarter AI Agents with Frameworks
-	Enterprise Gen AI with Embabel
-	How to build your own fun and absurd pair programmer
